#Расчёт градиентов

Задача:
1. В файле с данными гидрологических зондирований рассчитать значение производных для температуры воды (столбец Temperature), как функции от глубины (столбец Depth).

Для каждой точки измерения (столбец Station) имеем функцию T=T(h), распределение температуры по глубине.

2. Рассчитать производную 3 способами: разностной аппроксимацией вперед, назад и центральной.
3. Для каждой станции определить глубину, на которой начинается слой скачка (термоклин) и толщину этого слоя.

Выходные данные — таблица со столбцами: "N станции", "Глубина начала слоя термолина", "Толщина слоя термоклина".
Например, 18001;27.2;18.73

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# представление данных типа float (8 знаков после запятой)
pd.set_option('display.float_format', lambda x: '%.8f' % x)

In [5]:
data = pd.read_csv('hydrological_sounding_data.csv', sep=';', header=0)

In [6]:
data = data.sort_values(["Station", "Depth [m]"])
data = data[data['Depth [m]'] > 2]

In [7]:
stations = data['Station'].unique()

In [17]:
# функция нахождения центральной, правой, левой производных
def get_derivatives(station_data):
  y_i_0 = station_data['Temperature [degrees_C]'].shift(-1)
  y_i_1 = station_data['Temperature [degrees_C]']
  y_i_2 = station_data['Temperature [degrees_C]'].shift(1)
  x_i_0 = station_data["Depth [m]"].shift(-1)
  x_i_1 = station_data["Depth [m]"]
  x_i_2 = station_data["Depth [m]"].shift(1)

  derivative_forward = (y_i_0 - y_i_1)/(x_i_0 - x_i_1)
  derivative_backward = (y_i_1 - y_i_2)/(x_i_1 - x_i_2)
  derivative_central = (y_i_0 - y_i_2)/(x_i_0 - x_i_2)

  return (derivative_forward, derivative_backward, derivative_central)

In [13]:
data["derivative_forward"] = np.nan
data["derivative_backward"] = np.nan
data["derivative_central"] = np.nan

In [14]:
result_table = []

In [18]:
for station in stations:
    station_data = data[data['Station'] == station].copy()

    data.loc[data['Station'] == station, "Temperature [degrees_C]"] = station_data["Temperature [degrees_C]"]

    if station_data['Depth [m]'].max() < 75:
        station_data = station_data[station_data['Depth [m]'] < 40]
    elif station_data['Depth [m]'].max() < 110:
        station_data = station_data[station_data['Depth [m]'] < 60]

    max_depth = station_data['Depth [m]'].max()
    depths = station_data['Depth [m]'].values
    temperatures = station_data['Temperature [degrees_C]'].values

    # расчёт производных
    derivative_forward, derivative_backward, derivative_central = get_derivatives(station_data)
    station_data["derivative_forward"] = derivative_forward
    station_data["derivative_backward"] = derivative_backward
    station_data["derivative_central"] = derivative_central

    # сохранение производных в общий data frame
    data.loc[data['Station'] == station, "derivative_forward"] = station_data["derivative_forward"]
    data.loc[data['Station'] == station, "derivative_backward"] = station_data["derivative_backward"]
    data.loc[data['Station'] == station, "derivative_central"] = station_data["derivative_central"]

    threshold = 0.55 # порог для модуля производной

    if station > 18105:
        threshold = 0.08
    else:
        threshold = 0.55

    indices = abs(station_data["derivative_central"].values) > threshold

    if indices.any():
        start_depth = station_data["Depth [m]"][indices].iloc[0] # начало скачка
        end_depth = station_data["Depth [m]"][indices].iloc[-1]

        thickness = end_depth - start_depth # толщина термоклина

        result_table.append(f"{station};{start_depth:.2f};{thickness:.2f}")

<ipython-input-18-c9784a22091f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_data["derivative_forward"] = derivative_forward
<ipython-input-18-c9784a22091f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_data["derivative_backward"] = derivative_backward
<ipython-input-18-c9784a22091f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [19]:
results_table = pd.DataFrame([r.split(';') for r in result_table], columns=['Станция', 'Глубина термоклина (м)', 'Толщина термоклина (м)'])
data_copy = data.drop(columns=['Cruise', 'Date', 'Conductivity', 'Latitude [degrees_north]',  'Longitude [degrees_east]',   'Pressure', 'Bot. Depth [m]'])

print(results_table.to_string(index=False))

Станция Глубина термоклина (м) Толщина термоклина (м)
  18001                   6.12                  26.82
  18002                   4.34                  31.64
  18003                   5.29                  32.86
  18004                   6.32                  31.22
  18005                   6.33                  24.67
  18006                   7.20                  27.39
  18007                   5.88                  26.21
  18008                   7.20                  24.82
  18009                   5.72                  21.10
  18010                   5.89                  21.41
  18011                   7.64                  49.93
  18012                   8.40                  34.19
  18013                   7.75                  45.94
  18014                   8.06                  19.59
  18015                   5.77                  21.12
  18016                   7.16                  45.84
  18017                   8.37                  18.59
  18018                   7.